In [57]:

import itertools

import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite, Reaction
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)
t_model = cobra.io.read_sbml_model(model_file_path)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Model load time: 61.957249 seconds


In [58]:

for rx in t_model.exchanges:
    m = list(rx.metabolites.keys())[0]
    boundary_met = Metabolite(m.id[:-4] + 'x[x]', formula=m.formula, name=' '.join(m.name.split(' ')[:-1]) + ' [Boundary]', compartment='x')
    rx.add_metabolites({boundary_met: 1})

print(len(t_model.exchanges))

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


0


In [33]:
t = t_model.metabolites.get_by_id('m02560x[x]')
print(t.formula)
print(t.name)
print(t.reactions)

r1 = model.reactions.get_by_id('HMR_0015')
r2 = model.reactions.get_by_id('HMR_9033')
print(r1)
print(r2)

CHO2R
NEFA blood pool in [Boundary]
frozenset({<Reaction HMR_9033 at 0x253014a1b80>})
HMR_0015: m02560s[s] --> 0.0048 m00003s[s] + 0.0016 m00008s[s] + 0.0016 m00010s[s] + 0.0001 m00017s[s] + 0.0001 m00019s[s] + 0.0001 m00021s[s] + 0.0001 m00094s[s] + 0.0001 m00104s[s] + 0.0001 m00111s[s] + 0.0001 m00114s[s] + 0.0001 m00115s[s] + 0.0001 m00117s[s] + 0.0001 m00128s[s] + 0.0001 m00132s[s] + 0.0001 m00135s[s] + 0.0001 m00260s[s] + 0.0001 m00265s[s] + 0.0001 m00315s[s] + 0.0001 m00341s[s] + 0.0001 m01197s[s] + 0.0001 m01207s[s] + 0.0001 m01235s[s] + 0.0001 m01238s[s] + 0.0017 m01291s[s] + 0.0137 m01362s[s] + 0.0007 m01373s[s] + 0.0005 m01432s[s] + 0.0001 m01582s[s] + 0.0002 m01583s[s] + 0.0001 m01584s[s] + 0.0001 m01585s[s] + 0.0046 m01689s[s] + 0.0025 m01696s[s] + 0.0019 m01741s[s] + 0.0011 m01771s[s] + 0.0001 m01778s[s] + 0.002 m01784s[s] + 0.0048 m01932s[s] + 0.0001 m02053s[s] + 0.0034 m02344s[s] + 0.0012 m02385s[s] + 0.071 m02387s[s] + 0.0005 m02389s[s] + 0.0056 m02456s[s] + 0.0005 m024

In [173]:
# Import table
tasks = pd.read_table('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv')

print(tasks.head())

   id                                    description  shouldFail  printFluxes  \
0  ER  Aerobic rephosphorylation of ATP from glucose           0            0   
1  ER               Aerobic rephosphorylation of GTP           0            0   
2  ER               Aerobic rephosphorylation of CTP           0            0   
3  ER               Aerobic rephosphorylation of UTP           0            0   
4  BS                          ATP de novo synthesis           0            0   

   comments                        inputs     LBin                 UBin  \
0       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
1       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
2       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
3       NaN             'O2[s]glucose[s]'      0,0            1000,1000   
4       NaN  'O2[s]glucose[s]NH3[s]Pi[s]'  0,0,0,0  1000,1000,1000,1000   

                outputs  LBout           UBout  \
0        'H2

In [174]:
# Formating data
tasks['LBin'] = tasks['LBin'].apply(lambda x: x.split(','))
tasks['LBout'] = tasks['LBout'].apply(lambda x: x.split(','))
tasks['UBin'] = tasks['UBin'].apply(lambda x: x.split(','))
tasks['UBout'] = tasks['UBout'].apply(lambda x: x.split(','))

d = ']'
tasks['inputs'] = tasks['inputs'].apply(lambda x: [e+d for e in x[1:-1].split(d)][0:-1])
tasks['outputs'] = tasks['outputs'].apply(lambda x: [e+d for e in x[1:-1].split(d)][0:-1])
tasks['equations'] = tasks['equations'].apply(str)
print(tasks.head())

   id                                    description  shouldFail  printFluxes  \
0  ER  Aerobic rephosphorylation of ATP from glucose           0            0   
1  ER               Aerobic rephosphorylation of GTP           0            0   
2  ER               Aerobic rephosphorylation of CTP           0            0   
3  ER               Aerobic rephosphorylation of UTP           0            0   
4  BS                          ATP de novo synthesis           0            0   

   comments                              inputs          LBin  \
0       NaN                 [O2[s], glucose[s]]        [0, 0]   
1       NaN                 [O2[s], glucose[s]]        [0, 0]   
2       NaN                 [O2[s], glucose[s]]        [0, 0]   
3       NaN                 [O2[s], glucose[s]]        [0, 0]   
4       NaN  [O2[s], glucose[s], NH3[s], Pi[s]]  [0, 0, 0, 0]   

                       UBin                   outputs      LBout  \
0              [1000, 1000]          [H2O[s], CO2[s]] 

In [81]:
comps = {'s': 'Extracellular',
'p': 'Peroxisome',
'm': 'Mitochondria',
'c': 'Cytosol',
'l': 'Lysosome',
'r': 'Endoplasmic reticulum',
'g': 'Golgi apparatus',
'n': 'Nucleus',
'i': 'Inner mitochondria',
'x': 'Boundary'}

In [175]:
met_list = []

for i, data in tasks.iterrows():
    l = []
    if data['equations'] != 'nan':
        l = data['equations'].split(' ')
        l = [item for item in l if item[-1] == ']']

    met_list = met_list + data['inputs'] + data['outputs'] + l


met_list = list(set(met_list))
print(len(met_list))


120


In [176]:
mets = t_model.metabolites

met_ids = {}
for met in met_list:


    if met[:-3] == 'ALLMETSIN':
        met_ids[met] = 'ALLMETSIN'

    elif met[:-3] == 'ALLMETS':
        met_ids[met] = 'ALLMETS'

    else:
        comp = met[-2]
        temp = met[:-3] + ' [{0}]'.format(comps[comp])
        for m in mets:
            if m.name == temp:
                met_ids[met] = m
                break
        else:
            # Failed to find
            print(temp)
            print(met)


print(len(met_ids))

120


In [177]:
tasks.at[56, 'outputs']= [tasks.at[56, 'outputs'][1]] + tasks.at[56, 'inputs']
tasks.at[56, 'LBout']= [tasks.at[56, 'LBout'][1]] + tasks.at[56, 'LBin']
tasks.at[56, 'UBout']= [tasks.at[56, 'UBout'][1]] + tasks.at[56, 'UBin']

print(tasks.loc[56].at['outputs'])
print(len(tasks.loc[56].at['LBout']))
print(len(tasks.loc[56].at['UBout']))

['biomass[s]', 'arginine[s]', 'histidine[s]', 'lysine[s]', 'methionine[s]', 'phenylalanine[s]', 'tryptophan[s]', 'tyrosine[s]', 'alanine[s]', 'glycine[s]', 'serine[s]', 'threonine[s]', 'aspartate[s]', 'glutamate[s]', 'asparagine[s]', 'glutamine[s]', 'isoleucine[s]', 'leucine[s]', 'proline[s]', 'valine[s]', 'cysteine[s]', 'thiamin[s]', 'hypoxanthine[s]', 'folate[s]', 'biotin[s]', 'pantothenate[s]', 'choline[s]', 'inositol[s]', 'nicotinamide[s]', 'pyridoxine[s]', 'riboflavin[s]', 'thymidine[s]', 'aquacob(III)alamin[s]', 'lipoic acid[s]', 'glucose[s]', 'sulfate[s]', 'linoleate[s]', 'linolenate[s]', 'O2[s]', 'H2O[s]', 'retinoate[s]', 'Fe2+[s]', 'Pi[s]', 'alpha-tocopherol[s]', 'gamma-tocopherol[s]']
45
45


In [178]:
rx_list = []
in_list = []
out_list = []
for ind, data in tasks.iterrows():

    rx_in = Reaction('ess_{0}_in'.format(ind+1))
    rx_out = Reaction('ess_{0}_out'.format(ind+1))

    for i, name, ml, lbs, ubs in zip([1, -1], ['in', 'out'], [data.inputs, data.outputs], [data.LBin, data.LBout], [data.UBin, data.UBout]):
        rxl = []
        for j, m, lb, ub in zip( range(len(ml)),ml, lbs, ubs):
            rx = Reaction('ess_{0}_{1}_{2}'.format(ind+1, name, j))
            rx.add_metabolites({met_ids[m]: i})
            rx.lower_bound = float(lb)
            rx.upper_bound = float(ub)
            rxl.append(rx)
        if name == 'in':
            in_list.append(rxl)
        else:
            out_list.append(rxl)

    if data.equations != 'nan':
        t = [ [ met_ids[subsub] for subsub in sub.split(' ') if len(subsub)>1] for sub in data.equations.split('=') ]
        d = {}

        for i, ml in zip([-1, 1], t):
            for m in ml:
                d[m] = i

        rx = Reaction('ess_{0}'.format(ind+1))
        rx.add_metabolites(d)
        rx.lower_bound = float(data.LBequ)
        rx.upper_bound = float(data.UBequ)
        rx.name = data.description

        rx_list.append(rx)

    else:
        rx_list.append('nan')
task_list = pd.DataFrame(list(zip(in_list, out_list, rx_list)), columns=['in_rx', 'out_rx', 'equ'])

In [179]:
print(t_model.optimize())

<Solution 0.000 at 0x253a7d555e0>


In [226]:
n=0
sol = cobra.Solution
res = []
for ind, data in task_list.iterrows():

    n+=1
    if n>60:
        break
    #print(data.equ)


    with t_model:
        
        for rx in data.in_rx + data.out_rx:
            t_model.add_reaction(rx)

        if data.equ != 'nan':
            t_model.add_reaction(data.equ)

        for gene_id in ["ENSG00000165449", "ENSG00000100714"]:
            t_model.genes.get_by_id(gene_id).knock_out()

        sol = t_model.optimize()

    res.append(sol.objective_value)

C:\Users\Sigve\miniconda3\envs\Master\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [228]:
l = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,1,1]
l2 = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,1,1]
l3 = [1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1]
res2 = [0 if r is None else 1 for r in res]
res_df = pd.DataFrame(list(zip(res2, l3)))
print(res_df)

    0  1
0   1  1
1   1  1
2   1  1
3   1  1
4   0  0
5   1  1
6   0  0
7   1  1
8   0  0
9   1  1
10  0  0
11  1  1
12  1  1
13  1  1
14  1  1
15  1  1
16  1  1
17  1  1
18  1  1
19  1  1
20  1  1
21  1  1
22  1  1
23  1  1
24  1  1
25  1  1
26  1  1
27  1  1
28  1  1
29  1  1
30  1  1
31  1  1
32  1  1
33  1  1
34  1  1
35  1  1
36  1  1
37  1  1
38  1  1
39  1  1
40  1  1
41  1  1
42  1  1
43  1  1
44  1  1
45  1  1
46  1  1
47  1  1
48  1  1
49  1  1
50  1  1
51  1  1
52  1  1
53  0  0
54  0  0
55  0  0
56  1  1


In [210]:
print(res[50])

None


In [225]:
print(sol)
print(task_list.iloc[36, 0])
print(task_list.iloc[36, 1])


res_2 = sol.fluxes

print(res_2[res_2 != 0])

<Solution infeasible at 0x25308513b50>
[<Reaction ess_37_in_0 at 0x253a7075f70>, <Reaction ess_37_in_1 at 0x25301258220>, <Reaction ess_37_in_2 at 0x253012580a0>]
[<Reaction ess_37_out_0 at 0x25301258580>, <Reaction ess_37_out_1 at 0x25301258280>, <Reaction ess_37_out_2 at 0x25301259bb0>, <Reaction ess_37_out_3 at 0x25301259610>]
Series([], Name: fluxes, dtype: float64)


In [224]:
print(tasks.iloc[56, 7])
print(task_list.iloc[56, 0])
print(task_list.iloc[56, 1])

['1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000', '1000']
[<Reaction ess_57_in_0 at 0x253012812e0>, <Reaction ess_57_in_1 at 0x253012748e0>, <Reaction ess_57_in_2 at 0x253012744c0>, <Reaction ess_57_in_3 at 0x253012746a0>, <Reaction ess_57_in_4 at 0x25301274820>, <Reaction ess_57_in_5 at 0x25301274d00>, <Reaction ess_57_in_6 at 0x25301274ca0>, <Reaction ess_57_in_7 at 0x25301274e20>, <Reaction ess_57_in_8 at 0x25301274f70>, <Reaction ess_57_in_9 at 0x25301274b80>, <Reaction ess_57_in_10 at 0x25301274400>, <Reaction ess_57_in_11 at 0x25301274520>, <Reaction ess_57_in_12 at 0x25301274a90>, <Reaction ess_57_in_13 at 0x25301274160>, <Reaction ess_57_in_14 at 0x25301274f10>, <Reaction ess_57_in_15 at 